In [6]:
# ==== EXPORT SUBJECTS & SCALER (no model training) ====
from pathlib import Path
import json, joblib
import numpy as np
import pandas as pd

# ---- Config (giữ nguyên như bạn đang dùng) ----
DATA_XLSX   = Path("Data_clean/Data_subject_complete.xlsx")  # có cột 'split'
SCALER_PATH = Path("2/scaler.joblib")                        # nơi lưu scaler
SUBJECTS_JSON = Path("3/subjects.json")                      # nơi lưu danh sách môn

# ---- Load data ----
df = pd.read_excel(DATA_XLSX)

# Lấy các cột số làm "subjects"
subject_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if not subject_cols:
    raise ValueError("Không tìm thấy cột số nào trong file dữ liệu. Hãy kiểm tra DATA_XLSX.")

# Chỉ dùng TRAIN để tính scaler (best practice)
if "split" not in df.columns:
    raise ValueError("Thiếu cột 'split' trong dữ liệu. Cần có để tách train/val/test.")
df_tr = df[df["split"] == "train"].reset_index(drop=True)
if df_tr.empty:
    raise ValueError("Tập TRAIN rỗng. Hãy kiểm tra giá trị cột 'split'.")

# ---- Compute z-score stats theo TRAIN ----
train_means = df_tr[subject_cols].mean(axis=0)
train_stds  = df_tr[subject_cols].std(axis=0).replace(0, 1.0)  # tránh chia cho 0

# ---- Save artifacts ----
SCALER_PATH.parent.mkdir(parents=True, exist_ok=True)
Path(SUBJECTS_JSON).parent.mkdir(parents=True, exist_ok=True)

joblib.dump({"means": train_means.to_dict(), "stds": train_stds.to_dict()}, SCALER_PATH)
Path(SUBJECTS_JSON).write_text(json.dumps(subject_cols, ensure_ascii=False, indent=2), encoding="utf-8")

print("✅ Exported:")
print("  🧪 scaler:", SCALER_PATH.resolve())
print("  📜 subjects:", Path(SUBJECTS_JSON).resolve())
print(f"  (#subjects = {len(subject_cols)})")


✅ Exported:
  🧪 scaler: C:\Users\vuman\Desktop\AI_Project\Final in HUST\Project\training-find-score-ee\2\scaler.joblib
  📜 subjects: C:\Users\vuman\Desktop\AI_Project\Final in HUST\Project\training-find-score-ee\3\subjects.json
  (#subjects = 26)


In [7]:
# Cell A — Train GGM (Ledoit-Wolf / GraphicalLassoCV) & export
from pathlib import Path
import numpy as np
import pandas as pd
import joblib, json

from sklearn.covariance import LedoitWolf  # nhanh, ổn định
# from sklearn.covariance import GraphicalLassoCV  # nếu muốn sparse graph

DATA_XLSX  = Path("Data_clean/Data_subject_complete.xlsx")   # có 'split'
SCALER_P   = Path("2/scaler.joblib")                         
SUBJECTS_P = Path("3/subjects.json")                         
OUT_GGM    = Path("models_streamlit/EE2_ggm.joblib")

# Load data + artifacts
df        = pd.read_excel(DATA_XLSX)
subjects  = json.loads(Path(SUBJECTS_P).read_text(encoding="utf-8"))
scaler    = joblib.load(SCALER_P)
means     = pd.Series(scaler["means"])
stds      = pd.Series(scaler["stds"]).replace(0, 1.0)

# Lấy TRAIN và z-score
df_tr = df[df["split"] == "train"].reset_index(drop=True)
X_tr  = df_tr[subjects].copy()
X_std = (X_tr - means) / stds
X_std = X_std.fillna(0.0).values  # điền mean=0 sau z-score

# Ước lượng covariance
# Cách 1: LedoitWolf (khuyến nghị, nhanh)
est = LedoitWolf().fit(X_std)
cov = est.covariance_

# (Tuỳ chọn) Cách 2: GraphicalLassoCV (chậm hơn, ra precision thưa)
# est = GraphicalLassoCV().fit(X_std)
# cov = est.covariance_
# precision = est.precision_

# Lưu artifacts
OUT_GGM.parent.mkdir(parents=True, exist_ok=True)
ggm_art = {
    "cov": cov,                    # đủ để làm conditional prediction
    # "precision": precision,      # nếu dùng GraphicalLassoCV
    "subjects": subjects,
    "train_means": means.to_dict(),
    "train_stds": stds.to_dict(),
}
joblib.dump(ggm_art, OUT_GGM)
print("✅ Saved GGM to:", OUT_GGM.resolve())


✅ Saved GGM to: C:\Users\vuman\Desktop\AI_Project\Final in HUST\Project\training-find-score-ee\models_streamlit\EE2_ggm.joblib
